In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("davilsena/ckdataset")

print("Path to dataset files:", path)

100%|██████████| 2.48M/2.48M [00:00<00:00, 156MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/davilsena/ckdataset/versions/2


In [2]:
import numpy as np
#load the dataset
def load_dataset(path):
    # Placeholder function to load dataset
    # Replace this with actual loading and preprocessing code
    images = np.random.randn(1000, 64, 64, 3)  # Dummy data
    labels = np.random.randint(0, 8, 1000)     # Dummy labels (8 emotion classes)
    images = (images - 127.5) / 127.5          # Normalize to [-1, 1]
    return images, labels

images, labels = load_dataset('/root/.cache/kagglehub/datasets/davilsena/ckdataset/versions/2')

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Input,
    Dense,
    LeakyReLU,
    Reshape,
    Flatten,
    Conv2D,
    Conv2DTranspose,
    Dropout,
    Concatenate,
    GlobalAveragePooling2D,
    Layer
)
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
import numpy as np

def build_generator(latent_dim, num_classes):
    noise_input = Input(shape=(latent_dim,), name="noise_input")
    label_input = Input(shape=(1,), name="label_input")

    # Embed label into the same dimension as noise
    label_embedding = Dense(latent_dim, activation="relu")(label_input)

    # Combine noise and embedded label
    merged_input = Concatenate()([noise_input, label_embedding])

    # Fully connected layer
    x = Dense(128 * 8 * 8, activation="relu")(merged_input)
    x = Reshape((8, 8, 128))(x)

    # Upsampling with Conv2DTranspose
    x = Conv2DTranspose(128, kernel_size=4, strides=2, padding="same", activation="relu")(x)
    x = Conv2DTranspose(64, kernel_size=4, strides=2, padding="same", activation="relu")(x)
    # Adjust the upsampling layers to output the desired shape (32, 32, 3)
    x = Conv2DTranspose(64, kernel_size=4, strides=2, padding="same", activation="relu")(x)
    x = Conv2D(3, kernel_size=3, padding="same", activation="tanh")(x)  # Output layer with 3 channels and tanh activation


    # Remove unnecessary resizing and ResNet50 part

    return Model([noise_input, label_input], x, name="Generator") # Return x directly










In [4]:
def build_discriminator(image_shape, num_classes):
    image_input = Input(shape=image_shape, name="image_input")
    label_input = Input(shape=(1,), name="label_input")

    # Embed label into the image dimensions
    label_embedding = Dense(np.prod(image_shape))(label_input)
    label_embedding = Reshape(image_shape)(label_embedding)

    # Combine image and label
    merged_input = Concatenate(axis=-1)([image_input, label_embedding])

    # ResNet50 for feature extraction
    resnet_base = ResNet50(weights="imagenet", include_top=False, input_shape=image_shape)
    for layer in resnet_base.layers:
        layer.trainable = False  # Freeze ResNet50 layers

    # Extract ResNet features
    resnet_features = resnet_base(image_input)

    # Flatten VGG features for concatenation
    flat_features = Flatten()(resnet_features)

    # Embed the label input
    label_embedding = Dense(np.prod(flat_features.shape[1:]))(label_input)
    label_embedding = LeakyReLU(alpha=0.2)(label_embedding)

    # Combine flat VGG features and label embedding
    merged = Concatenate()([flat_features, label_embedding])

    # Fully connected layers
    x = Dense(128)(merged)
    x = LeakyReLU(alpha=0.2)(x)
    validity = Dense(1, activation="sigmoid", name="validity")(x)

    return Model([image_input, label_input], validity, name="Discriminator")

In [5]:
# Parameters for models
latent_dim = 100
num_classes = 10
image_shape = (64, 64, 3)

# Instantiate models
generator = build_generator(latent_dim, num_classes)
discriminator = build_discriminator(image_shape, num_classes)

# Print model summaries
print(generator.summary())
print(discriminator.summary())

/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "Generator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ label_input (InputLayer)  │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ noise_input (InputLayer)  │ (None, 100)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 100)            │            200 │ label_input[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 200)            │              0 │ noise_input[0][0],     │
│                           │                        │                │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 8192)           │      1,646,592 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape (Reshape)         │ (None, 8, 8, 128)      │              0 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose          │ (None, 16, 16, 128)    │        262,272 │ reshape[0][0]          │
│ (Conv2DTranspose)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose_1        │ (None, 32, 32, 64)     │        131,136 │ conv2d_transpose[0][0] │
│ (Conv2DTranspose)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose_2        │ (None, 64, 64, 64)     │         65,600 │ conv2d_transpose_1[0]… │
│ (Conv2DTranspose)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 64, 64, 3)      │          1,731 │ conv2d_transpose_2[0]… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 2,107,531 (8.04 MB)

 Trainable params: 2,107,531 (8.04 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ image_input (InputLayer)  │ (None, 64, 64, 3)      │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ label_input (InputLayer)  │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ resnet50 (Functional)     │ (None, 2, 2, 2048)     │     23,587,712 │ image_input[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 8192)           │         16,384 │ label_input[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 8192)           │              0 │ resnet50[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu (LeakyReLU)   │ (None, 8192)           │              0 │ dense_3[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_2             │ (None, 16384)          │              0 │ flatten[0][0],         │
│ (Concatenate)             │                        │                │ leaky_re_lu[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 128)            │      2,097,280 │ concatenate_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_1 (LeakyReLU) │ (None, 128)            │              0 │ dense_4[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ validity (Dense)          │ (None, 1)              │            129 │ leaky_re_lu_1[0][0]    │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 25,701,505 (98.04 MB)

 Trainable params: 2,113,793 (8.06 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

None


In [6]:
#cGAN Model
def build_cgan(generator, discriminator, latent_dim, ttur_gen_lr, ttur_disc_lr): # Added ttur_gen_lr as an argument
    discriminator.compile(optimizer=Adam(ttur_disc_lr, 0.5), loss="binary_crossentropy", metrics=["accuracy"])
    discriminator.trainable = False

    noise_input = Input(shape=(latent_dim,), name="noise_input")
    label_input = Input(shape=(1,), name="label_input")
    generated_img = generator([noise_input, label_input])
    validity = discriminator([generated_img, label_input])

    cgan = Model([noise_input, label_input], validity, name="cGAN")
    cgan.compile(optimizer=Adam(ttur_gen_lr, 0.5), loss="binary_crossentropy") # Now using the correct learning rate
    return cgan

In [6]:
#load the dataset
def load_dataset(path):
    # Placeholder function to load dataset
    # Replace this with actual loading and preprocessing code
    images = np.random.randn(1000, 64, 64, 3)  # Dummy data
    labels = np.random.randint(0, 8, 1000)     # Dummy labels (8 emotion classes)
    images = (images - 127.5) / 127.5          # Normalize to [-1, 1]
    return images, labels

images, labels = load_dataset('/root/.cache/kagglehub/datasets/nelgiriyewithana/emotions/versions/1')

In [7]:
import numpy as np
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Embedding, Concatenate
from tensorflow.keras.optimizers import Adam



batch_size = 64
epochs = 5

# Build and compile models
generator = build_generator(latent_dim, num_classes)
discriminator = build_discriminator(image_shape, num_classes)
discriminator.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['accuracy'])

ttur_gen_lr = 0.0002  # Example value, adjust as needed
ttur_disc_lr = 0.0002  # Example value, adjust as needed

cgan = build_cgan(generator, discriminator,latent_dim, ttur_gen_lr, ttur_disc_lr)
cgan.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy')

# Custom Data Generator for cGAN Training
class CGANDataGenerator(Sequence):
    def __init__(self, images, labels, batch_size, latent_dim, num_classes):
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.latent_dim = latent_dim
        self.num_classes = num_classes

    def __len__(self):
        return len(self.images) // self.batch_size

    def __getitem__(self, index):
        # Real images and labels for discriminator
        idx = np.random.randint(0, self.images.shape[0], self.batch_size // 2)
        real_images = self.images[idx]
        real_labels = self.labels[idx]

        # Fake images and labels for discriminator
        noise = np.random.normal(0, 1, (self.batch_size // 2, self.latent_dim))
        fake_labels = np.random.randint(0, self.num_classes, self.batch_size // 2)
        fake_images = generator.predict([noise, fake_labels])

        # Inputs and labels for discriminator
        discriminator_x = np.concatenate([real_images, fake_images])
        discriminator_y = np.concatenate([np.ones((self.batch_size // 2, 1)), np.zeros((self.batch_size // 2, 1))])
        discriminator_labels = np.concatenate([real_labels, fake_labels])

        return ([discriminator_x, discriminator_labels], discriminator_y)

# Dummy dataset for demonstration
images = np.random.rand(1000, *image_shape)
labels = np.random.randint(0, num_classes, 1000)

data_generator = CGANDataGenerator(images, labels, batch_size, latent_dim, num_classes)

# Training loop
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    for batch_idx in range(len(data_generator)):
        # 1. Train Discriminator
        discriminator_x, discriminator_y = data_generator[batch_idx]
        d_loss = discriminator.train_on_batch(discriminator_x, discriminator_y)

        # 2. Train Generator (via the combined model)
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        random_labels = np.random.randint(0, num_classes, batch_size)
        g_loss = cgan.train_on_batch([noise, random_labels], np.ones((batch_size, 1)))

    # Print progress

        print(f"Epoch: {epoch + 1}/{epochs}, Discriminator Loss: {d_loss[0]}, Generator Loss: {g_loss}")
# Save the models
generator.save("generator.h5")
discriminator.save("discriminator.h5")
cgan.save("cgan.h5")


Epoch 1/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch: 1/5, Discriminator Loss: 0.8840395212173462, Generator Loss: [array(0.8840395, dtype=float32), array(0.8840395, dtype=float32), array(0.5, dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Epoch: 1/5, Discriminator Loss: 0.8852442502975464, Generator Loss: [array(0.88524425, dtype=float32), array(0.88524425, dtype=float32), array(0.5, dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


Epoch: 1/5, Discriminator Loss: 0.8862809538841248, Generator Loss: [array(0.88628095, dtype=float32), array(0.88628095, dtype=float32), array(0.5, dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Epoch: 1/5, Discriminator Loss: 0.8873397707939148, Generator Loss: [array(0.8873398, dtype=float32), array(0.8873398, dtype=float32), array(0.5, dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Epoch: 1/5, Discriminator Loss: 0.888497531414032, Generator Loss: [array(0.88849753, dtype=float32), array(0.88849753, dtype=float32), array(0.5, dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Epoch: 1/5, Discriminator Loss: 0.8906843662261963, Generator Loss: [array(0.89068437, dtype=float32), array(0.89068437, dtype=float32), array(0.5, dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Epoch: 1/5, Discriminator Loss: 0.8931645154953003, Generator Loss: [array(0.8931645, dtype=float32), array(0.8931645, dtype=float32), array(0.5, dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/

Epoch: 5/5, Discriminator Loss: 1.1469229459762573, Generator Loss: [array(1.146923, dtype=float32), array(1.146923, dtype=float32), array(0.5, dtype=float32)]


In [9]:
!pip install tensorflow scipy


import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from scipy.stats import entropy


def calculate_inception_score(generator, latent_dim, num_classes, n_images=5000, batch_size=128):
    """
    Calculate the Inception Score for a GAN model.

    Args:
    - generator: The trained generator model.
    - latent_dim: Latent space dimension used by the generator.
    - num_classes: Number of classes for the conditional GAN.
    - n_images: Total number of images to generate for evaluation.
    - batch_size: Batch size for generating images.

    Returns:
    - mean IS score and standard deviation.
    """
    # Load pre-trained InceptionV3 model
    inception_model = InceptionV3(include_top=True, weights="imagenet")
    inception_model = Model(inputs=inception_model.input, outputs=inception_model.get_layer("avg_pool").output)

    # Generate images and their corresponding labels
    n_batches = n_images // batch_size
    preds = []
    for _ in range(n_batches):
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        labels = np.random.randint(0, num_classes, batch_size)
        generated_images = generator.predict([noise, labels], verbose=0)

        # Resize images to (299, 299, 3) for InceptionV3 compatibility
        resized_images = tf.image.resize(generated_images, (299, 299)).numpy()

        # Normalize images to [-1, 1] -> [0, 1]
        normalized_images = (resized_images + 1) / 2.0
        preds.append(inception_model.predict(normalized_images, verbose=0))

    preds = np.concatenate(preds, axis=0)

    # Compute probabilities and the Inception Score
    scores = []
    for pred in preds:
        # Reshape pred to have shape (1, -1) to calculate entropy along axis=1
        pred = pred.reshape(1, -1)
        p_y = np.mean(pred, axis=0)
        kl_div = entropy(pred, p_y, axis=1)
        scores.append(np.exp(np.mean(kl_div)))

    return np.mean(scores), np.std(scores)





In [19]:
mean_is, std_is = calculate_inception_score(generator, latent_dim, num_classes, n_images, batch_size)
print(f"Mean Inception Score: {mean_is:.4f}") # Print the mean IS score
print(f"Inception Score: {mean_is:.4f} ± {std_is:.4f}")

Mean Inception Score: 1.0000
Inception Score: 1.0000 ± 0.0000
